In [30]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels
import seaborn as sns
import os
import time
%matplotlib inline

## Data read 

In [31]:
train = pd.read_csv('/media/vsppraneeth/01D3522569C0B1A0/WSDM_v2/train_v2.csv') # Train.csv

In [32]:
train.head(10)

,msno,is_churn
0,ugx0CjOMzazClkFzU2xasmDZaoIqOUAZPsH1q0teWCg=,1
1,f/NmvEzHfhINFEYZTR05prUdr+E+3+oewvweYz9cCQE=,1
2,zLo9f73nGGT1p21ltZC3ChiRnAVvgibMyazbCxvWPcg=,1
3,8iF/+8HY8lJKFrTc7iR9ZYGCG2Ecrogbc2Vy5YhsfhQ=,1
4,K6fja4+jmoZ5xG6BypqX80Uw/XKpMgrEMdG2edFOxnA=,1
5,ibIHVYBqxGwrSExE63/omeDD99M5vYB3CN2HzkEY+eM=,1
6,kVmM8X4iBPCOfK/m1l3KR7LjFoxD1EcKYCc76F5IAWw=,1
7,moRTKhKIDvb+C8ZHOgmaF4dXMLk0jOn65d7a8tQ2Eds=,1
8,dW/tPZMDh2Oz/ksduEctJbsz0MXw3kay/1AlZCq3EbI=,1
9,otEcMhAX3mU4gumUSogqgteN3oaNmhdmTkoof2iRYEE=,1


In [33]:
train['msno'] = train['msno'].astype(str)
train['is_churn'] = train['is_churn'].astype('category')

In [34]:
train.shape

(970960, 2)

In [35]:
members = pd.read_csv('/media/vsppraneeth/01D3522569C0B1A0/WSDM_v2/members_v3.csv') # Members.csv

In [36]:
members.head(10)

,msno,city,bd,gender,registered_via,registration_init_time
0,Rb9UwLQTrxzBVwCB6+bCcSQWZ9JiNLC9dXtM1oEsZA8=,1,0,NaN,11,20110911
1,+tJonkh+O1CA796Fm5X60UMOtB6POHAwPjbTRVl/EuU=,1,0,NaN,7,20110914
2,cV358ssn7a0f7jZOwGNWS07wCKVqxyiImJUX6xcIwKw=,1,0,NaN,11,20110915
3,9bzDeJP6sQodK73K5CBlJ6fgIQzPeLnRl0p5B77XP+g=,1,0,NaN,11,20110915
4,WFLY3s7z4EZsieHCt63XrsdtfTEmJ+2PnnKLH5GY4Tk=,6,32,female,9,20110915
5,yLkV2gbZ4GLFwqTOXLVHz0VGrMYcgBGgKZ3kj9RiYu8=,4,30,male,9,20110916
6,jNCGK78YkTyId3H3wFavcBLDmz7pfqlvCfUKf4G1Lw4=,1,0,NaN,7,20110916
7,WH5Jq4mgtfUFXh2yz+HrcTXKS4Oess4k4W3qKolAeb0=,5,34,male,9,20110916
8,tKmbR4X5VXjHmxERrckawEMZ4znVy1lAQIR1vV5rdNk=,5,19,male,9,20110917
9,I0yFvqMoNkM8ZNHb617e1RBzIS/YRKemHO7Wj13EtA0=,13,63,male,9,20110918


In [37]:
members['registration_init_time'] = pd.to_datetime(members['registration_init_time'],format = '%Y%m%d')
members['msno'] = members['msno'].astype(str)
members['city'] = members['city'].astype('category')
members['gender'] = members['gender'].astype('category')
members['registered_via'] = members['registered_via'].astype('category')
# Filling missing values :
members['gender'] = members['gender'].replace(np.nan,'missing',regex = True)
bd_fill = int(np.mean(members.bd[(members.bd > 0) & (members.bd < 101)]))
members.bd[(members.bd < 1) | (members.bd > 100)] = bd_fill

/home/vsppraneeth/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [38]:
members.head(5)

,msno,city,bd,gender,registered_via,registration_init_time
0,Rb9UwLQTrxzBVwCB6+bCcSQWZ9JiNLC9dXtM1oEsZA8=,1,29,missing,11,2011-09-11
1,+tJonkh+O1CA796Fm5X60UMOtB6POHAwPjbTRVl/EuU=,1,29,missing,7,2011-09-14
2,cV358ssn7a0f7jZOwGNWS07wCKVqxyiImJUX6xcIwKw=,1,29,missing,11,2011-09-15
3,9bzDeJP6sQodK73K5CBlJ6fgIQzPeLnRl0p5B77XP+g=,1,29,missing,11,2011-09-15
4,WFLY3s7z4EZsieHCt63XrsdtfTEmJ+2PnnKLH5GY4Tk=,6,32,female,9,2011-09-15


In [39]:
Final_Dataframe = train.merge(members, how = 'left', on = 'msno')

In [40]:
Final_Dataframe.shape

(970960, 7)

In [41]:
del members

In [42]:
transactions = pd.read_csv('/media/vsppraneeth/01D3522569C0B1A0/WSDM_v2/Transactions_Month_Grouped_Updated.csv') # Transactons.csv

In [43]:
transactions = transactions.drop(['Unnamed: 0|201609', 'Unnamed: 0|201610', 'Unnamed: 0|201611',
       'Unnamed: 0|201612', 'Unnamed: 0|201701', 'Unnamed: 0|201702',
       'Unnamed: 0|201703'], axis = 1)

In [44]:
transactions.head()

,msno,payment_plan_days|201609,payment_plan_days|201610,payment_plan_days|201611,payment_plan_days|201612,payment_plan_days|201701,payment_plan_days|201702,payment_plan_days|201703,plan_list_price|201609,plan_list_price|201610,...,discount|201701,discount|201702,discount|201703,is_discount|201609,is_discount|201610,is_discount|201611,is_discount|201612,is_discount|201701,is_discount|201702,is_discount|201703
0,+++FOrTS7ab3tIgIh8eWwX4FqRv8w/FoiOuyXsFvphY=,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,+++IZseRRiQS9aaSkH6cMYU6bGDcxUieAi/tH67sC5s=,0.0,395.0,0.0,0.0,0.0,0.0,0.0,0.0,1599.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,+++hVY1rZox/33YtvDgmKA2Frg/2qhkz12B9ylCvh8o=,0.0,0.0,30.0,30.0,30.0,30.0,30.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,+++l/EXNMLTijfLBa8p2TUVVVp2aFGSuUI/h7mLmthw=,30.0,30.0,30.0,30.0,30.0,30.0,30.0,149.0,149.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,+++snpr7pmobhLKUgSHTv/mpkqgBT0tQJ0zQj6qKrqc=,30.0,30.0,30.0,30.0,30.0,30.0,30.0,149.0,149.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [45]:
Final_Dataframe = Final_Dataframe.merge(transactions, how = 'left', on = 'msno')

In [46]:
Final_Dataframe.shape

(970960, 42)

In [47]:
del transactions

In [48]:
user_logs = pd.read_csv('/media/vsppraneeth/E6A2B390A2B36429/WSDM/User_Logs_Month_Grouped.csv')

In [49]:
Final_Dataframe = Final_Dataframe.merge(user_logs, how = 'left', on = 'msno')

In [50]:
del user_logs

In [51]:
user_logs_2 = pd.read_csv('/media/vsppraneeth/01D3522569C0B1A0/WSDM_v2/Download_MG_UserLogs.csv')

In [52]:
Final_Dataframe = Final_Dataframe.merge(user_logs_2, how = 'left', on = 'msno')

In [53]:
del user_logs_2

In [54]:
Final_Dataframe.head(10)

,msno,is_churn,city,bd,gender,registered_via,registration_init_time,payment_plan_days|201609,payment_plan_days|201610,payment_plan_days|201611,...,total_secs|201612,total_secs|201701,total_secs|201702,num_25|201703,num_50|201703,num_75|201703,num_985|201703,num_100|201703,num_unq|201703,total_secs|201703
0,ugx0CjOMzazClkFzU2xasmDZaoIqOUAZPsH1q0teWCg=,1,5,28.0,male,3,2013-12-23,0.0,0.0,0.0,...,0.000,0.000,12472.612,186.0,23.0,13.0,10.0,318.0,348.0,80598.557
1,f/NmvEzHfhINFEYZTR05prUdr+E+3+oewvweYz9cCQE=,1,13,20.0,male,3,2013-12-23,0.0,0.0,0.0,...,0.000,0.000,18474.223,0.0,4.0,2.0,0.0,26.0,30.0,6986.509
2,zLo9f73nGGT1p21ltZC3ChiRnAVvgibMyazbCxvWPcg=,1,13,18.0,male,3,2013-12-27,0.0,30.0,30.0,...,101482.189,93466.669,68146.893,239.0,57.0,32.0,22.0,205.0,432.0,67810.467
3,8iF/+8HY8lJKFrTc7iR9ZYGCG2Ecrogbc2Vy5YhsfhQ=,1,1,29.0,missing,7,2014-01-09,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,K6fja4+jmoZ5xG6BypqX80Uw/XKpMgrEMdG2edFOxnA=,1,13,35.0,female,7,2014-01-25,30.0,30.0,30.0,...,37836.665,968.371,45495.387,9.0,7.0,4.0,4.0,962.0,548.0,239882.241
5,ibIHVYBqxGwrSExE63/omeDD99M5vYB3CN2HzkEY+eM=,1,22,29.0,missing,3,2014-01-26,30.0,30.0,30.0,...,38186.547,16939.815,69685.131,6.0,1.0,1.0,4.0,37.0,48.0,10110.665
6,kVmM8X4iBPCOfK/m1l3KR7LjFoxD1EcKYCc76F5IAWw=,1,1,29.0,missing,7,2014-01-29,30.0,30.0,30.0,...,10101.114,4829.124,0.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,moRTKhKIDvb+C8ZHOgmaF4dXMLk0jOn65d7a8tQ2Eds=,1,9,28.0,female,3,2014-02-02,0.0,0.0,0.0,...,221228.068,268627.665,252772.439,78.0,20.0,30.0,16.0,1023.0,758.0,290831.795
8,dW/tPZMDh2Oz/ksduEctJbsz0MXw3kay/1AlZCq3EbI=,1,13,21.0,female,9,2014-02-12,0.0,120.0,0.0,...,109212.392,148005.774,153234.568,291.0,87.0,43.0,38.0,658.0,915.0,179129.056
9,otEcMhAX3mU4gumUSogqgteN3oaNmhdmTkoof2iRYEE=,1,13,29.0,missing,7,2014-02-28,30.0,30.0,60.0,...,84931.310,115509.132,70589.899,456.0,34.0,11.0,20.0,365.0,605.0,97597.540


In [55]:
Write_Dataframe = pd.get_dummies(Final_Dataframe, columns=['gender', 'registered_via', 'city'])
Write_Dataframe = Write_Dataframe.drop(['registration_init_time','msno'], axis = 1)
Write_Dataframe.head(10)

,is_churn,bd,payment_plan_days|201609,payment_plan_days|201610,payment_plan_days|201611,payment_plan_days|201612,payment_plan_days|201701,payment_plan_days|201702,payment_plan_days|201703,plan_list_price|201609,...,city_13,city_14,city_15,city_16,city_17,city_18,city_19,city_20,city_21,city_22
0,1,28.0,0.0,0.0,0.0,0.0,0.0,30.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
1,1,20.0,0.0,0.0,0.0,0.0,0.0,30.0,30.0,0.0,...,1,0,0,0,0,0,0,0,0,0
2,1,18.0,0.0,30.0,30.0,30.0,60.0,0.0,150.0,0.0,...,1,0,0,0,0,0,0,0,0,0
3,1,29.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
4,1,35.0,30.0,30.0,30.0,30.0,90.0,30.0,30.0,99.0,...,1,0,0,0,0,0,0,0,0,0
5,1,29.0,30.0,30.0,30.0,30.0,30.0,30.0,30.0,149.0,...,0,0,0,0,0,0,0,0,0,1
6,1,29.0,30.0,30.0,30.0,30.0,30.0,30.0,30.0,99.0,...,0,0,0,0,0,0,0,0,0,0
7,1,28.0,0.0,0.0,0.0,0.0,0.0,0.0,410.0,0.0,...,0,0,0,0,0,0,0,0,0,0
8,1,21.0,0.0,120.0,0.0,0.0,30.0,0.0,120.0,0.0,...,1,0,0,0,0,0,0,0,0,0
9,1,29.0,30.0,30.0,60.0,30.0,30.0,30.0,30.0,149.0,...,1,0,0,0,0,0,0,0,0,0


In [56]:
Write_Dataframe.bd = Write_Dataframe.bd.fillna(Write_Dataframe.bd.median())
Write_Dataframe = Write_Dataframe.fillna(0)

In [57]:
Write_Dataframe.head(10)

,is_churn,bd,payment_plan_days|201609,payment_plan_days|201610,payment_plan_days|201611,payment_plan_days|201612,payment_plan_days|201701,payment_plan_days|201702,payment_plan_days|201703,plan_list_price|201609,...,city_13,city_14,city_15,city_16,city_17,city_18,city_19,city_20,city_21,city_22
0,1,28.0,0.0,0.0,0.0,0.0,0.0,30.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
1,1,20.0,0.0,0.0,0.0,0.0,0.0,30.0,30.0,0.0,...,1,0,0,0,0,0,0,0,0,0
2,1,18.0,0.0,30.0,30.0,30.0,60.0,0.0,150.0,0.0,...,1,0,0,0,0,0,0,0,0,0
3,1,29.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
4,1,35.0,30.0,30.0,30.0,30.0,90.0,30.0,30.0,99.0,...,1,0,0,0,0,0,0,0,0,0
5,1,29.0,30.0,30.0,30.0,30.0,30.0,30.0,30.0,149.0,...,0,0,0,0,0,0,0,0,0,1
6,1,29.0,30.0,30.0,30.0,30.0,30.0,30.0,30.0,99.0,...,0,0,0,0,0,0,0,0,0,0
7,1,28.0,0.0,0.0,0.0,0.0,0.0,0.0,410.0,0.0,...,0,0,0,0,0,0,0,0,0,0
8,1,21.0,0.0,120.0,0.0,0.0,30.0,0.0,120.0,0.0,...,1,0,0,0,0,0,0,0,0,0
9,1,29.0,30.0,30.0,60.0,30.0,30.0,30.0,30.0,149.0,...,1,0,0,0,0,0,0,0,0,0


In [58]:
Write_Dataframe.to_csv('/media/vsppraneeth/01D3522569C0B1A0/WSDM/ML_Train_updated.csv')